# Binary classification with Scikit-Learn

In [ ]:
import sklearn

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

## Our task

UCI ML Breast Cancer Wisconsin (Diagnostic) dataset, which contains 569 biopsies of a suspect breast cancer that can be either *malignat* or *benign*.
Sample features are descriptors of the cell nuclei present in the image of each biopsy.

Details here https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
X, y = load_breast_cancer(return_X_y=True)

In [ ]:
X.shape,  y.shape

In [ ]:
y_classes, y_counts = np.unique(y, return_counts=True)

In [ ]:
plt.bar(['malignant (0)', 'benign (1)'], y_counts)

### Construct selection and test sets

Now we create an arbitrarily split in order to have a selection set and a test set for the next experiments. Usually those splits are given by the task, e.g. ML Cup dataset for students and blind test set for the teacher.

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
X, y = load_breast_cancer(return_X_y=True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=100, stratify=y, shuffle=True, random_state=42)

In [ ]:
X_dev.shape, X_test.shape

## Evaluation metrics

We'll now see some evaluation metrics for a (binary) classifier, using a dummy baseline as an example

### A baseline classifier

An extremely naive baseline classifier returns always the *most frequent class* as prediction:

In [ ]:
np.unique(y_dev, return_counts=True)

In [ ]:
y_pred = np.ones_like(y_dev)

In [ ]:
y_test_pred = np.ones_like(y_test)

### Evaluating the baseline

In [ ]:
from sklearn.metrics import *

Accuracy: ratio of correct predictions

In [ ]:
accuracy_score(y_test, y_test_pred)

Balanced accuracy: ratio of correct predictions in each class weighted by class frequency

In [ ]:
balanced_accuracy_score(y_test, y_test_pred)

The confusion matrix lets us see how the classifier is behaving on the different classes

In [ ]:
cm = confusion_matrix(y_test, y_test_pred)

ConfusionMatrixDisplay(cm).plot()

Let's try with a *random guess* classifier instead:

In [ ]:
y_test_pred = np.random.randint(2, size=y_test.shape)

In [ ]:
cm = confusion_matrix(y_test, y_test_pred)

ConfusionMatrixDisplay(cm).plot()

For unbalanced classes where one class is of particular interest, such as malignant cancer in our case, other metrics can be useful that consider *precision* and *recall* of the classifier:

![image.svg](https://upload.wikimedia.org/wikipedia/commons/2/26/Precisionrecall.svg)

In [ ]:
precision_score(y_test, y_test_pred, pos_label=0), recall_score(y_test, y_test_pred, pos_label=0)

In [ ]:
precision_score(y_test, y_test_pred, pos_label=1), recall_score(y_test, y_test_pred, pos_label=1)

F1 is the harmonic mean of precision and recall

In [ ]:
f1_score(y_test, y_test_pred, pos_label=0), f1_score(y_test, y_test_pred, pos_label=1)

In the medical field, the receiver operating characteristic (ROC) curve is used to display the trade-off between false positive rate ("false alarms") and true positive rate

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_test_pred, pos_label=0)
RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

In [ ]:
roc_auc_score(y_test, y_test_pred)

## Data transformations

Models can benefit from a normalization of feature values, e.g. in k-NN when computing distances between sample vectors $\mathbf{x} = [x_1, x_2]$, so distances in one feature with values in a smaller scale (e.g. $x_1 \in [0,1]$) will not be trumped by distances in another feature with larger scale (e.g. $x_2 \in [0, 1000]$).

In [ ]:
from sklearn.preprocessing import *

In [ ]:
plt.hist(X_dev[:,0], alpha=.5, label='x0')
plt.hist(X_dev[:,5], alpha=.5, label='x1')
plt.legend(loc='upper right')
plt.plot()

- Rescale values between minimimum and maximum:

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X_dev)

In [ ]:
X_dev_scaled = scaler.transform(X_dev)

In [ ]:
plt.hist(X_dev_scaled[:,0], alpha=.5, label='x0')
plt.hist(X_dev_scaled[:,1], alpha=.5, label='x1')
plt.legend(loc='upper right')
plt.plot()

- Normalize values with mean and standard deviation:

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(X_dev)

In [ ]:
X_dev_scaled = scaler.transform(X_dev)

In [ ]:
plt.hist(X_dev_scaled[:,0], alpha=.5, label='x0')
plt.hist(X_dev_scaled[:,1], alpha=.5, label='x1')
plt.legend(loc='upper right')
plt.plot()

## Nearest neighbour

k-NN returns the majority class of the $k$ samples that are closest to $\mathbf{x}$.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
model = KNeighborsClassifier(n_neighbors=3,  # number of neighbours
                             weights='uniform',  # or weight proportional to the inverse of the distance
                             metric='minkowski',  # or other user-defined distances
                             p=2)  # p-norm for 'minkowski' metric

By "Minkowski" distance here it is denoted the $p$-norm of a vector $\mathbf{x} - \mathbf{y}$, defined as $\lVert\mathbf{x} - \mathbf{y}\rVert_p = \sqrt[p]{\sum_k (x_k - y_k)^p}$

In [ ]:
model.fit(X_dev, y_dev)

In [ ]:
accuracy_score(y_test, model.predict(X_test))

You can also combine a feature scaler with a classifier model in a single pipeline:

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
model = make_pipeline(MinMaxScaler(), KNeighborsClassifier())

In [ ]:
model.fit(X_dev, y_dev)

In [ ]:
accuracy_score(y_test, model.predict(X_test))

Parameters can be accessed as `<step>.<parameter>` (useful for model selection, as we're about to see)

In [ ]:
model.named_steps

In [ ]:
model.named_steps['kneighborsclassifier'].n_neighbors

## Model selection

Scikit-learns offers helper classes to automaticall split a dataset in k-folds and to perform model parameter selection

### K-Fold cross-validation

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
kfold.split(X_dev, y_dev)

A generator which returns pairs of `(train_fold_indices, validation_fold_indices)`:

In [ ]:
list(kfold.split(X_dev, y_dev))

In case of classification, it could be useful to use `StratifiedKFold` in order to keep the same class ratio in all splits

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

For a double cross-validation, just nest a two k-fold splits in a loop:

In [ ]:
outer_kfold = KFold(n_splits=10)
inner_kfold = KFold(n_splits=5)

for selection_indices, test_indices in outer_kfold.split(X, y):
    for train_indices, validation_indices in inner_kfold.split(X[selection_indices], y[selection_indices]):
        ...

### Validation curve

We can now analyze the impact of the parameter $k$ in a k-NN classifier on validation accuracy:

In [ ]:
from sklearn.model_selection import validation_curve

In [ ]:
k = np.arange(1, 10)

In [ ]:
train_scores, validation_scores = validation_curve(KNeighborsClassifier(),
                                                   X_dev, y_dev,
                                                   param_name='n_neighbors',
                                                   param_range=k,
                                                   scoring='balanced_accuracy',
                                                   cv=kfold)

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
valid_scores_mean = np.mean(validation_scores, axis=1)
valid_scores_std = np.std(validation_scores, axis=1)

plt.title("Validation Curve with k-NN")
plt.xlabel(r"$k$")
plt.ylabel("Score")
plt.ylim(0.75, 1.05)
lw = 2
plt.plot(
    k, train_scores_mean, label="Training score", color="darkorange", lw=lw
)
plt.fill_between(
    k,
    train_scores_mean - train_scores_std,
    train_scores_mean + train_scores_std,
    alpha=0.2,
    color="darkorange",
    lw=lw,
)
plt.plot(
    k, valid_scores_mean, label="Cross-validation score", color="navy", lw=lw
)
plt.fill_between(
    k,
    valid_scores_mean - valid_scores_std,
    valid_scores_mean + valid_scores_std,
    alpha=0.2,
    color="navy",
    lw=lw,
)
plt.legend(loc="lower left")
plt.show()

### Hyper-parameter search

Instead of manually searching the combination of parameters, Scikit-learns offers classes that automate hyper-parameter selection

- Generation of all parameter combinations

In [ ]:
from sklearn.model_selection import ParameterGrid

In [ ]:
param_values = {'n_neighbors': [1, 2, 3, 4, 5],
                'p': [1, 2]}

In [ ]:
list(ParameterGrid(param_values))

- Grid search

`GridSearchCV` performs an exhaustive search of model hyper-parameters that maximize the average score on a cross-validation split

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
hp_search = GridSearchCV(KNeighborsClassifier(),
                         param_grid=param_values,
                         scoring='balanced_accuracy',
                         refit=True,
                         cv=kfold)

In [ ]:
hp_search.fit(X_dev, y_dev)

In the grid search object you can access different statistics of the model selection such as paramters and scores, best estimator, training times, etc

In [ ]:
hp_search.cv_results_

In [ ]:
hp_search.best_estimator_

In [ ]:
hp_search.best_score_

In [ ]:
hp_search.best_params_

We can now evaluate the best model on the test set:

In [ ]:
hp_search.score(X_test, y_test)

- Random search

Since the hyper-parameter space grows exponentially with the number of parameters, another strategy is to sample uniformly this space as a trade-off between computation time and hyper-parameter exploration

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

You can also choose the distribution of parameter, instead of just enumerating them, e.g. for continously valued params

In [ ]:
from scipy.stats import uniform

In [ ]:
hp_rsearch = RandomizedSearchCV(KNeighborsClassifier(),
                                param_distributions={'n_neighbors': [1, 2, 3, 4, 5],
                                                     'p': uniform(loc=1, scale=2)},
                                n_iter=5,  # how many hyper-parameter combinations to sample
                                scoring='balanced_accuracy',
                                refit=True,
                                cv=kfold)

In [ ]:
hp_rsearch.fit(X_dev, y_dev)

In [ ]:
hp_rsearch.best_params_

In [ ]:
hp_rsearch.score(X_test, y_test)

## Linear models

This class of models is essentially in the form $\hat{y} = f(\mathbf{w}^T \mathbf{x} + w_0)$, with paramters $\mathbf{w}$ and $w_0$ to be trained. These models return a *score* as prediction, and the decision function is $\mathrm{score} > \mathrm{threshold}$.
Training is essentially a regression on classes values (e.g. 1-hot encodings in case of multi-class problems), $\min_{w} || X w - y||_2^2$ plus regularization.

### Ridge classifier

A linear model $\hat{y} = f(\mathbf{w}^T \mathbf{x} + w_0)$ which is trained by least-squares regression $\min_{w} || X w - y||_2^2 + \alpha ||w||_2^2$ with target values of classes as $\pm 1$. Decision function is the sign of the prediction score $\hat{y}$.

In [ ]:
from sklearn.linear_model import RidgeClassifier

In [ ]:
clf = RidgeClassifier(alpha=1.0,  # regularization parameter
                      class_weight='balanced',  # use this to balence unbalanced classes 
                      solver='auto')  # choose solving method (e.g. SVD, SGD, ...)

In [ ]:
clf.fit(X_dev, y_dev)

In [ ]:
clf.score(X_test, y_test)

Other details here https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html

### Logistic regression

Model that tries to learn the positive class probability $P(y_i=1|X_i)$ as $\hat{p}(X_i) = \operatorname{expit}(X_i w + w_0) = \frac{1}{1 + \exp(-X_i w - w_0)}$, by minimizing the loss $\min_{w} C \sum_{i=1}^n \left(-y_i \log(\hat{p}(X_i)) - (1 - y_i) \log(1 - \hat{p}(X_i))\right) + r(w)$.

The term $r(w)$ is a regularization penalty (e.g. L1, L2 norms, ...); higher $C$ ⇒ lower regularization.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression(penalty='l2',  # type of penalty
                         C=1.0,
                         solver='lbfgs',  # type of solver; depends also on the type of penalty, see documentation
                         max_iter=10000,  # see documentation for other solver parameters...
                         class_weight='balanced',  # use this to balence unbalanced classes
                         verbose=1)

In [ ]:
clf.fit(X_dev, y_dev)

In [ ]:
clf.score(X_test, y_test)

Since the classifier returns scores, when can see the ROC curve for different threshold values

In [ ]:
fpr, tpr, _ = roc_curve(y_test, clf.predict_proba(X_test)[:,0], pos_label=0)
RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

### Other linear models

Other variants of linear models are distinguished by the type of regularization that is applied in error minimization:
- `Lasso`: Weights have L1 regularization to favor sparsity, 
- `ElasticNet`: Weights have both L1 and L2 regularization,
- ...

## Neural networks

A neural network classifier, where you can choose the hidden layers and their units, the training procedure (SGD, LBGFS, Adam), regularization, etc. The loss to be minimized is *cross-entropy loss*.

The L2 regularization parameter is $\alpha$, larger ⇒ more regularization.

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
nn = MLPClassifier(hidden_layer_sizes=(8,),  # input and output layer sizes automatically selected by fit()
                   activation='tanh',  # activation function
                   solver='sgd',  # {‘lbfgs’, ‘sgd’, ‘adam’}
                   alpha=1e-2,  # L2 regularization (is divided by #samples)
                   max_iter=100,  # epochs
                   tol=1e-6,  # training loss improvement tolerance (training is halted if training loss stops improving)
                   n_iter_no_change=50,  # patience for training loss improvement
                   batch_size=32,
                   shuffle=True,  # reshuffle samples between epochs
                   learning_rate='constant',  # can also be adaptive
                   learning_rate_init=1e-3,  # (initial) learning rate
                   momentum=0.9,
                   nesterovs_momentum=False,  # if you want to use Nesterov’s momentum, only for SGD
                   early_stopping=False,  # if you want to use a hold-out set for early stopping
                   verbose=True)

nn.fit(X_dev, y_dev)

You can access learning curve and other training statistics in the `MLPClassifier` object

In [ ]:
plt.plot(nn.loss_curve_)

In [ ]:
nn.score(X_test, y_test)

## Support vector machines

Support vector machines for classification (C-SVM).

This class solves the soft-margin problem:
$\begin{align}\begin{aligned}\min_ {w, b, \zeta} \frac{1}{2} w^T w + C \sum_{i=1}^{n} \zeta_i\\\begin{split}\textrm {subject to } & y_i (w^T \phi (x_i) + b) \geq 1 - \zeta_i,\\
& \zeta_i \geq 0, i=1, ..., n\end{split}\end{aligned}\end{align}$
where $C$ controls the strenght of regularization: larger $C$ ⇒ smaller regularization.

In the dual form, the kernel trick is applied in the scalar products $K(x_i, x_j) = \phi (x_i)^T \phi (x_j)$.

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm = SVC(C=1.0,
          kernel='linear',
          class_weight='balanced',  # use this to balence unbalanced classes
          verbose=True)

For the linear SVM, the class `LinearSVC` can also be used, where the kernel trick is not applied.

In [ ]:
svm.fit(X_dev, y_dev)

You can access statistics concerning the fit, such as number of support vectors per class:

In [ ]:
svm.n_support_

In [ ]:
svm.score(X_test, y_test)

## Decision trees

Decision trees which learn a hiearachical decision function by successive refinements of splits. Model complexity is controlled by tree depth or samples per node.

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
dt = DecisionTreeClassifier(max_depth=3)

In [ ]:
dt.fit(X_dev, y_dev)

In [ ]:
dt.score(X_test, y_test)

In [ ]:
plot_tree(dt)

### Exercise

Perform a model selection on one of these models (e.g. SVM with regularization, kernel type; NN with different hidden units, regularization, learning rate, epochs, ...), and report test **balanced accuracy**, (average) **validation accuracy**, and **test AUC**. You must use the test splits fixed above, otherwise you're free to perform the model selection as you wish (number of folds, grid or random search, ...). Apply also feature rescaling, if you deem it appropriate.

Submit the results here: https://tinyurl.com/ml2025-sklearn

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
X, y = load_breast_cancer(return_X_y=True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=100, stratify=y, shuffle=True, random_state=42)